Grid post-processing of Met.no data using SHyFT API
=========

### This notebook gives an example of Met.no data post-processing to correct temperature forecasts based on comparison to observations. The following steps are described:
1. **Loading required python modules and setting path to SHyFT installation**
2. **Create synthetic data for observation time-series**
3. **Create synthetic data for forecast time-series**
4. **Calculate correction bias between observations and forecasts**
5. **Apply bias correction to forecast time-series**
6. **Plot the results**

### 1. Loading required python modules and setting path to SHyFT installation

In [ ]:
# first you should import the third-party python modules which you'll use later on
# the first line enables that figures are shown inline, directly in the notebook
%pylab inline
import os
from os import path
import sys
from matplotlib import pyplot as plt

In [ ]:
# set the path for your shyft build
# this should point to the directory that is created
# when you clone shyft, assuming you have built shyft
# there and not installed it to your system python
shyft_path = os.path.abspath("../../../shyft")
sys.path.insert(0, shyft_path)

# you could achieve the same by setting a PYTHONPATH

In [ ]:
# once the shyft_path is set correctly, you should be able to import shyft modules
import shyft
from shyft import shyftdata_dir

# if you have problems here, it may be related to having your LD_LIBRARY_PATH
# pointing to the appropriate libboost_python libraries (.so files)
from shyft.repository.default_state_repository import DefaultStateRepository
from shyft.orchestration.configuration import yaml_configs
from shyft.orchestration.simulators.config_simulator import ConfigSimulator
from shyft import api

In [ ]:
# now you can access the api of shyft with tab completion and help, try this:

#help(api.GeoPoint) # remove the hashtag and run the cell to print the documentation of the api.GeoPoint class
#api. # remove the hashtag, set the pointer behind the dot and use 
      # tab completion to see the available attributes of the shyft api

### 2. Create synthetic data for observation time-series

In [ ]:
# Create time-axis starting 01.01.2016, sampled every hour for 10 days long
t0 = api.Calendar().time(2016, 1, 1)
ta = api.Timeaxis(t0, api.deltahours(1), 240)
ta2 = api.Timeaxis2(t0, api.deltahours(1), 240)

# Create a set of geo-locations at observation points (sparse set covering the forecast grid)
obs_locs = api.GeoPointVector()
obs_locs.append(api.GeoPoint( 100, 100,   10))
obs_locs.append(api.GeoPoint(5100, 100,  250))
obs_locs.append(api.GeoPoint( 100, 5100, 250))
obs_locs.append(api.GeoPoint(5100, 5100, 500))

# Create a set of time-series at observation points, having constant value
obs_set = api.GeoPointSourceVector()
for loc in obs_locs:
    obs_set.append(api.GeoPointSource(loc, api.Timeseries(ta2, fill_value=10)))

# Create a set of geo-locations at forecast points (evenly spaced grid of 10 x 10 km)
fc_locs = api.GeoPointVector()
for x in range(10):
    for y in range(10):
        fc_locs.append(api.GeoPoint(x*1000, y*1000, (x+y)*50))

# Spread the set of observations to forecast locations by ordinary Kriging
obs_grid = api.ordinary_kriging(obs_set, fc_locs, ta, api.OKParameter())

### 3. Create synthetic data for forecast time-series

In [ ]:
# Create an empty forecast grid
fc_grid = api.PrecipitationSourceVector()

# Create a synthetic bias time-serie
bias_ts = api.Timeseries(ta2, fill_value=1)

# Create forecast time-series by adding the synthetic bias to the observation time-series
for obs in obs_grid:
    fc_grid.append(api.PrecipitationSource(obs.mid_point(), obs.ts + bias_ts))

# Create a forecast set to observation locations by IDW transform
fc_set = api.idw_precipitation(fc_grid, obs_locs, ta, api.IDWPrecipitationParameter())

# Check that the grid mapping matches the observation locations (less than 5% error is fine)
is_good = abs(fc_set[0].ts.value(0) - fc_grid[0].ts.value(0)) < 0.05
print('Is mapping good? ' + str(is_good))
if not is_good:
    print('Forecast at forecast location = ' + str(fc_grid[0].ts.value(0)))
    print('Forecast at observation location = ' + str(fc_set[0].ts.value(0)))

### 4. Calculate correction bias between observations and forecasts

In [ ]:
# Create an empty bias set
bias_set = api.GeoPointSourceVector()

# Calculate the bias as a correction factor between observation and forecast timeseries at observation locations
for (obs, fc) in zip(obs_set, fc_set):
    bias_set.append(api.GeoPointSource(obs.mid_point(), obs.ts / fc.ts))
    
# Spread the bias from observation locations to forecast locations by ordinary kriging
bias_grid = api.ordinary_kriging(bias_set, fc_locs, ta, api.OKParameter())

print('The correction factor at the first forecast location is {0:.3f}'.format(bias_grid[0].ts.value(0)))

### 5. Apply bias correction to forecast time-series

In [ ]:
# Apply bias correction to forecast time-series at forecast locations
for (fc, bias) in zip(fc_grid, bias_grid):
    fc.ts = fc.ts * bias.ts

# Transform corrected forecasts to observation locations by IDW transform
fc_set = api.idw_precipitation(fc_grid, obs_locs, ta, api.IDWPrecipitationParameter())

# Check corrected forecast vs observation 
is_good = abs(fc_set[0].ts.value(0) - obs_set[0].ts.value(0)) < 0.05
print('Is correction good? ' + str(is_good))
if not is_good:
    print('Observation at observation location = ' + str(obs_set[0].ts.value(0)))
    print('Forecast at observation location = ' + str(fc_set[0].ts.value(0)))

### 6. Plot the results

In [ ]:
# Make a time-series plot
fig, ax = plt.subplots(figsize=(20, 10))
for i in range(len(bias_set)):
    timestamps = [datetime.datetime.utcfromtimestamp(p.start) for p in obs_set[i].ts.time_axis]
    ax.plot(timestamps, obs_set[i].ts.values, label = str(i+1) + ' Observation')
    ax.plot(timestamps, fc_set[i].ts.values, label = str(i+1) + ' Forecast')
    ax.plot(timestamps, bias_set[i].ts.values, label = str(i+1) + ' Bias')
fig.autofmt_xdate()
ax.legend(title='Precipitation')
ax.set_ylabel('Precipitation')

In [ ]:
# Make a scatter plot of grid temperature forecasts at ts.value(0)
x = [fc.mid_point().x for fc in fc_grid]
y = [fc.mid_point().y for fc in fc_grid]
fig, ax = plt.subplots(figsize=(10, 5))
temps = np.array([fc.ts.value(0) for fc in fc_grid])
plot = ax.scatter(x, y, c=temps, marker='s', vmin=0, vmax=12, s=500, lw=0)
plt.colorbar(plot).set_label('Precipitation')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15,5))
temps = np.array([fc.ts.value(0) for fc in fc_grid])
ax1.scatter(x, y, c=temps, marker='s', vmin=0, vmax=20, s=500, lw=0)
temps = np.array([obs.ts.value(0) for obs in bias_grid])
ax2.scatter(x, y, c=temps, marker='s', vmin=0, vmax=20, s=500, lw=0)
#plt.colorbar(plot).set_label('Precipitation')